In [ ]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset


#Data processing


***Reading the dataset***

In [ ]:
#Reading the csv
df = pd.read_csv('/content/drive/MyDrive/Collab/cyberbullying_tweets.csv', names=['text', 'cyberbullying_type'])


total_nulls = df.isnull().sum().sum()

print("Null data:", total_nulls)

# Making sure is a string (just in case)
df['cyberbullying_type'] = df['cyberbullying_type'].astype(str)

# Select 200 text for each label
df_sampled = df.groupby('cyberbullying_type').apply(lambda x: x.sample(n=min(len(x), 200), random_state=42)).reset_index(drop=True)

# Drop the categories we don't plan to use
df_sampled = df_sampled[~df_sampled['cyberbullying_type'].isin(['cyberbullying_type', 'other_cyberbullying'])]

Null data: 0


***Getting the labels***

In [ ]:
# Convert to categorical and set a index number for the selected labels
df_sampled['cyberbullying_type'] = df_sampled['cyberbullying_type'].astype('category')
df_sampled['label'] = df_sampled['cyberbullying_type'].cat.codes

# Crear el mapeo de categorías a códigos numéricos
category_mapping = dict(enumerate(df_sampled['cyberbullying_type'].cat.categories))

print("Ciberbullying numerical labels:")
for code, category in category_mapping.items():
    print(f"{code}: {category}")


Ciberbullying numerical labels:
0: age
1: ethnicity
2: gender
3: not_cyberbullying
4: religion


***Getting the train and test values***

In [ ]:
# Split data
train_df, test_df = train_test_split(df_sampled, test_size=0.2, random_state=42)

***Tokenization using GPT-2***

In [ ]:
# GPT-2 Tokenizator
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = '[PAD]'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

***Preparing the data in order to train and validate***

In [ ]:
class CyberbullyingDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }


***Calling the class we made***

In [ ]:
# Creating the datasets for the training and tests
train_dataset = CyberbullyingDataset(
    texts=train_df['text'].tolist(),
    labels=train_df['label'].tolist(),
    tokenizer=tokenizer
)

test_dataset = CyberbullyingDataset(
    texts=test_df['text'].tolist(),
    labels=test_df['label'].tolist(),
    tokenizer=tokenizer
)

***Instantiating the model***

In [ ]:
# Definir el modelo GPT-2 para clasificación de secuencias y ajustar los tokens especiales
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=df_sampled['cyberbullying_type'].nunique())
model.resize_token_embeddings(len(tokenizer))

# Establecer el token de padding en la configuración del modelo
model.config.pad_token_id = tokenizer.pad_token_id

# Mover el modelo al mismo dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

***Defining and training the model***

In [ ]:
# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',          # Directorio de salida para los checkpoints y los logs
    num_train_epochs=3,              # Número total de epochs de entrenamiento
    per_device_train_batch_size=8,   # Tamaño del batch de entrenamiento por dispositivo
    per_device_eval_batch_size=8,    # Tamaño del batch de evaluación por dispositivo
    warmup_steps=500,                # Número de pasos para el calentamiento del optimizador
    weight_decay=0.01,               # Factor de decaimiento para el ajuste de los pesos
    logging_dir='./logs',            # Directorio de logs
    evaluation_strategy='epoch'      # Evaluar el modelo al final de cada epoch
)


# Crear el objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Entrenar el modelo
trainer.train()

***Evaluation and proccess to save the trained model***

In [ ]:
# Evaluar el modelo en el conjunto de prueba
predictions = trainer.predict(test_dataset)
predicted_classes = np.argmax(predictions.predictions, axis=1)
true_classes = test_df['label'].values

# Mostrar métricas de clasificación
print(classification_report(true_classes, predicted_classes))

# Guardar el modelo entrenado
model.save_pretrained('./gpt2_cyberbullying_model')
tokenizer.save_pretrained('./gpt2_cyberbullying_tokenizer')

Mapeo de códigos numéricos a categorías de ciberacoso:
0: age
1: ethnicity
2: gender
3: not_cyberbullying
4: religion


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,2.297518
2,No log,1.557155
3,No log,1.237377


              precision    recall  f1-score   support

           0       0.61      0.96      0.75        28
           1       0.50      0.79      0.61        14
           2       0.17      0.20      0.18        10
           3       0.44      0.29      0.35        24
           4       1.00      0.25      0.40        24

    accuracy                           0.53       100
   macro avg       0.54      0.50      0.46       100
weighted avg       0.60      0.53      0.49       100



('./gpt2_cyberbullying_tokenizer/tokenizer_config.json',
 './gpt2_cyberbullying_tokenizer/special_tokens_map.json',
 './gpt2_cyberbullying_tokenizer/vocab.json',
 './gpt2_cyberbullying_tokenizer/merges.txt',
 './gpt2_cyberbullying_tokenizer/added_tokens.json')

***Using the model in inference***

In [ ]:
# Hacer inferencias con el modelo entrenado
text_to_predict = "man fuck lebron and Eli y'all just mad our owner a better gm then that dumb nigger y'all have in office"
encoded_text = tokenizer(text_to_predict, truncation=True, padding=True, return_tensors="pt")

# Mover los tensores a la misma GPU que el modelo si es necesario
encoded_text = {k: v.to(device) for k, v in encoded_text.items()}

logits = model(**encoded_text).logits
predicted_class = torch.argmax(logits, dim=1).item()

# Mostrar la clase predicha junto con su descripción
predicted_category = category_mapping[predicted_class]
print("Clase predicha:", predicted_class, "-", predicted_category)

Clase predicha: 0 - age
